# Final Project

# INGV - Volcanic Eruption Prediction

**Your name:** Carter Figgins

**A-Number:** A02243978



In [477]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob

device = torch.device("cuda")


In [478]:
# can import kaggle.json this way
# from google.colab import files
# uploaded = files.upload()


In [479]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !mkdir train
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle competitions download -c predict-volcanic-eruptions-ingv-oe -p ./train

# !unzip 1000015382.csv.zip



## 1. Read in Data

In [480]:
# get data from google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [481]:
# set up train data

path = "./drive/MyDrive"
train = pd.read_csv(path + "/train.csv")
segment_ids = train['segment_id']

count = 0

# for index, row in train.iterrows():
#   # print(index)
#   print(row['segment_id'])
#   count+=1
#   if count == 20:
#     break

# REMOVE?
# read in csv from test folder and put in dict with id as the key
train_sensors = []
train_times = []
for _, row in train.iterrows():
  s_id = str(row['segment_id'])
  data = pd.read_csv(path + "/train/" + s_id + ".csv")
  # normalizing data
  data = data.apply(lambda x: (x - x.mean()) / (x.std()))
  data = data.fillna(0)
  # data = data.head()
  data = data[:].values.astype(float)

  # REMOVE?
  # data = torch.utils.data.DataLoader(
  #     data,
  #     batch_size = batch_size,
  #     shuffle=True,
  #     num_workers=2,
  # )

  train_sensors.append(data)
  train_times.append(row['time_to_eruption'])
  # REMOVE?
  # train_sensors[s_id] = {'data': data, 'label': np.array(row['time_to_eruption'])}
  
  # smaller data for testing
  count+=1
  if count == 12:
    break


## 2. Define the Model

In [482]:
class Net (nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.linear = nn.Sequential(
        nn.Linear(10, 20),
        nn.ReLU(True),
        nn.Linear(20, 50),
        nn.ReLU(True),
        nn.Linear(50,25),
        nn.ReLU(True),
        nn.Linear(25,1)
    )


  def forward(self, x):
    x = self.linear(x)
    return x

## 3. Train Model

In [483]:
from torch.utils.data import Dataset
class Valcano_Dataset(Dataset):
    def __init__(self, train_features, train_labels):
        super().__init__()
        self.train_features = train_features
        self.train_labels = train_labels

    def __len__(self):
        return len(self.train_features)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return (self.train_features[idx], self.train_labels[idx])

train_data = Valcano_Dataset(train_sensors, train_times)

In [484]:
#Hyper-parameters
batch_size = 64
epochs = 5
learning_rate = 0.001
# Optimizer and loss function
MSE_loss = nn.MSELoss()
net = Net().cuda()
# net = net.float()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0005, weight_decay=1e-4)

In [488]:
for epoch in range(epochs):
  count = 0
  for i in range(len(train_data)):
    data , label = train_data[i]
    # label = train_times[i]
    data = data.astype(float)
    data = torch.from_numpy(data).float().cuda()
    label = label.astype(float).reshape((-1, 1))
    label = torch.from_numpy(np.array(label)).cuda()
    y_hat = net(data)
    print(y_hat.size())
    print(label.size())
    print(label)
    loss = MSE_loss(y_hat, label.float())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    
    count+=1
    if count == 1:
      break

torch.Size([60001, 1])
torch.Size([1, 1])
tensor([[12262005.]], device='cuda:0', dtype=torch.float64)
torch.Size([60001, 1])
torch.Size([1, 1])
tensor([[12262005.]], device='cuda:0', dtype=torch.float64)
torch.Size([60001, 1])
torch.Size([1, 1])
tensor([[12262005.]], device='cuda:0', dtype=torch.float64)
torch.Size([60001, 1])
torch.Size([1, 1])
tensor([[12262005.]], device='cuda:0', dtype=torch.float64)
torch.Size([60001, 1])
torch.Size([1, 1])
tensor([[12262005.]], device='cuda:0', dtype=torch.float64)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([60001, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [486]:
# read in all test files 
test_files = glob.glob(path + "/test/*.csv")
test_files

['./drive/MyDrive/test/585632599.csv',
 './drive/MyDrive/test/58542575.csv',
 './drive/MyDrive/test/586623109.csv',
 './drive/MyDrive/test/585794453.csv',
 './drive/MyDrive/test/585631329.csv',
 './drive/MyDrive/test/589599653.csv',
 './drive/MyDrive/test/589267026.csv',
 './drive/MyDrive/test/590127879.csv',
 './drive/MyDrive/test/587942731.csv',
 './drive/MyDrive/test/593905769.csv',
 './drive/MyDrive/test/591724983.csv',
 './drive/MyDrive/test/59115085.csv',
 './drive/MyDrive/test/591351810.csv',
 './drive/MyDrive/test/591748234.csv',
 './drive/MyDrive/test/595878150.csv',
 './drive/MyDrive/test/595300301.csv',
 './drive/MyDrive/test/595515594.csv',
 './drive/MyDrive/test/595948357.csv',
 './drive/MyDrive/test/596712964.csv',
 './drive/MyDrive/test/596902566.csv',
 './drive/MyDrive/test/596148775.csv',
 './drive/MyDrive/test/596188072.csv',
 './drive/MyDrive/test/597680020.csv',
 './drive/MyDrive/test/599282805.csv',
 './drive/MyDrive/test/599844877.csv',
 './drive/MyDrive/test/5989